#1

In [ ]:
import re
from typing import Dict, List, Tuple
from itertools import product

class Bid:
    def __init__(self, advertiser_id: int, empty: int, t: int, s: int, b: int):
        self.advertiser_id = advertiser_id
        self.empty = empty
        self.t = t
        self.s = s
        self.b = b

class TrueValuation:
    def __init__(self, advertiser_id: int, empty: int, t: int, s: int, b: int):
        self.advertiser_id = advertiser_id
        self.empty = empty

        self.t = t
        self.s = s
        self.b = b

class VCGAuction:
    def __init__(self, bids: List[Bid], true_valuations: List[TrueValuation]):
        self.bids = bids
        self.true_valuations = {tv.advertiser_id: tv for tv in true_valuations}

    def find_optimal_allocation(self) -> Tuple[Dict[int, str], int]:
        best_allocation = {}
        max_value = 0
        allocations = self.generate_allocations(len(self.bids))
        for allocation in allocations:
            allocation_dict = {self.bids[i].advertiser_id: bid for i, bid in enumerate(allocation)}
            if self.is_valid_allocation(allocation_dict):
                value = self.calculate_value(allocation_dict)
                if value > max_value or (value == max_value and self.prefers_both_slots(allocation_dict)):
                    max_value = value
                    best_allocation = allocation_dict

        return best_allocation, max_value

    def generate_allocations(self, num_bids: int) -> List[List[str]]:
        return list(product(['t', 's', 'b', ''], repeat=num_bids))

    def is_valid_allocation(self, allocation: Dict[int, str]) -> bool:
        t_count = sum(1 for bid in allocation.values() if bid in ['t', 'b'])
        s_count = sum(1 for bid in allocation.values() if bid in ['s', 'b'])
        return t_count <= 1 and s_count <= 1

    def calculate_value(self, allocation: Dict[int, str]) -> int:
        total_value = 0
        for bid in self.bids:
            if allocation[bid.advertiser_id] == 't':
                total_value += bid.t
            elif allocation[bid.advertiser_id] == 's':
                total_value += bid.s
            elif allocation[bid.advertiser_id] == 'b':
                total_value += bid.b
            else:  # ''
                total_value += bid.empty
        return total_value

    def prefers_both_slots(self, allocation: Dict[int, str]) -> bool:
        return any(slot == 'b' for slot in allocation.values())

    def calculate_payments(self, optimal_allocation: Dict[int, str]) -> Dict[int, int]:
        payments = {}
        total_value = self.calculate_value(optimal_allocation)

        for bid in self.bids:
            bids_without_i = [b for b in self.bids if b.advertiser_id != bid.advertiser_id]
            auction_without_i = VCGAuction(bids_without_i, [])
            allocation_without_i, value_without_i = auction_without_i.find_optimal_allocation()
            value_others_with_i = total_value - self.get_bid_value(bid, optimal_allocation[bid.advertiser_id])
            payments[bid.advertiser_id] = max(0, value_without_i - value_others_with_i)

        return payments

    def get_bid_value(self, bid: Bid, allocation: str) -> int:
        if allocation == 't':
            return bid.t
        elif allocation == 's':
            return bid.s
        elif allocation == 'b':
            return bid.b
        else:  # ''
            return bid.empty

    def calculate_utilities(self, allocation: Dict[int, str], payments: Dict[int, int]) -> Dict[int, int]:
        utilities = {}
        for bid in self.bids:
            true_value = self.get_true_value(bid.advertiser_id, allocation[bid.advertiser_id])
            utilities[bid.advertiser_id] = true_value - payments[bid.advertiser_id]
        return utilities

    def get_true_value(self, advertiser_id: int, allocation: str) -> int:
        true_valuation = self.true_valuations[advertiser_id]
        if allocation == 't':
            return true_valuation.t
        elif allocation == 's':
            return true_valuation.s
        elif allocation == 'b':
            return true_valuation.b
        else:  # ''
            return true_valuation.empty

    def run_auction(self) -> Tuple[Dict[int, str], Dict[int, int], Dict[int, int]]:
        optimal_allocation, _ = self.find_optimal_allocation()
        payments = self.calculate_payments(optimal_allocation)
        utilities = self.calculate_utilities(optimal_allocation, payments)
        return optimal_allocation, payments, utilities

def parse_input() -> Tuple[List[Bid], List[TrueValuation]]:
    bids = []
    true_valuations = []
    print("Enter the number of bids:")
    n = int(input().strip())
    print("Enter the bids in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            bids.append(Bid(*parts))

    print("Enter the true valuations in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            true_valuations.append(TrueValuation(*parts))

    return bids, true_valuations

def format_output(allocation: Dict[int, str], payments: Dict[int, int], utilities: Dict[int, int]) -> str:
    output = "Allocation:\n"
    output += str(allocation) + "\n"
    output += "Payment:\n"
    output += str(payments) + "\n"
    output += "Utilities:\n"
    output += str(utilities)
    return output

def calculate_revenue(payments: Dict[int, int]) -> int:
    return sum(payments.values())

if __name__ == "__main__":
    bids, true_valuations = parse_input()
    auction = VCGAuction(bids, true_valuations)
    allocation, payments, utilities = auction.run_auction()
    revenue = calculate_revenue(payments)
    print(format_output(allocation, payments, utilities))
    print(f"Auctioneer's Revenue: {revenue}")

Enter the number of bids:
3
Enter the bids in the format: AdvertiserID empty t s b
1 0 2 2 2
2 0 1  1 1
3 0 2 2 4
Enter the true valuations in the format: AdvertiserID empty t s b
1 0 2 2 2
2 0 1 1 1
3 0 2 2 4
Allocation:
{1: '', 2: '', 3: 'b'}
Payment:
{1: 0, 2: 0, 3: 3}
Utilities:
{1: 0, 2: 0, 3: 1}
Auctioneer's Revenue: 3


#2

In [ ]:

def parse_input(n: int) -> Tuple[List[Bid], List[TrueValuation]]:
    bids = []
    true_valuations = []
    print("Enter the bids in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            bids.append(Bid(*parts))

    print("Enter the true valuations in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            true_valuations.append(TrueValuation(*parts))

    return bids, true_valuations

def format_output(allocation: Dict[int, str], payments: Dict[int, int], utilities: Dict[int, int]) -> str:
    output = "Allocation:\n"
    output += str(allocation) + "\n"
    output += "Payment:\n"
    output += str(payments) + "\n"
    output += "Utilities:\n"
    output += str(utilities)
    return output

def calculate_revenue(payments: Dict[int, int]) -> int:
    return sum(payments.values())

def run_double_identity_auction(bids: List[Bid], true_valuations: List[TrueValuation], alpha: float) -> Tuple[Dict[int, str], Dict[int, int], Dict[int, int], float]:
    auction = VCGAuction(bids, true_valuations)
    allocation, payments, utilities = auction.run_auction()

    double_agent_id = bids[-1].advertiser_id
    utilities[double_agent_id] = utilities[bids[-2].advertiser_id] + utilities[bids[-1].advertiser_id] - alpha

    revenue = calculate_revenue(payments)
    return allocation, payments, utilities, revenue

if __name__ == "__main__":
    print("Enter the number of bidders:")
    n = int(input().strip())

    print("\nOriginal VCG Auction:")
    bids, true_valuations = parse_input(n)
    auction = VCGAuction(bids, true_valuations)
    allocation, payments, utilities = auction.run_auction()
    revenue = calculate_revenue(payments)
    print(format_output(allocation, payments, utilities))
    print(f"Auctioneer's Revenue: {revenue}")

    print("\nDouble Identity Scenario:")
    print("Enter the alpha value:")
    alpha = float(input().strip())

    print(f"Enter {n+1} bids for the double identity scenario:")
    double_bids, double_true_valuations = parse_input(n+1)

    d_allocation, d_payments, d_utilities, d_revenue = run_double_identity_auction(double_bids, double_true_valuations, alpha)
    print(format_output(d_allocation, d_payments, d_utilities))
    print(f"Auctioneer's Revenue: {d_revenue}")

    print("\nComparison:")
    original_utility = utilities[n]
    double_identity_utility = d_utilities[n+1]
    print(f"Original utility for bidder {n}: {original_utility}")
    print(f"Double identity utility for bidder {n}: {double_identity_utility}")
    print(f"Difference: {double_identity_utility - original_utility}")

Enter the number of bidders:
3

Original VCG Auction:
Enter the bids in the format: AdvertiserID empty t s b
1 0 0 0 15
2 0 0 0 12
3 0 10 0 10
Enter the true valuations in the format: AdvertiserID empty t s b
1 0 0 0 15
2 0 0 0 12
3 0 10 0 10
Allocation:
{1: 'b', 2: '', 3: ''}
Payment:
{1: 0, 2: 0, 3: 0}
Utilities:
{1: 15, 2: 0, 3: 0}
Auctioneer's Revenue: 0

Double Identity Scenario:
Enter the alpha value:
0.1
Enter 4 bids for the double identity scenario:
Enter the bids in the format: AdvertiserID empty t s b
1 0 0 0 15
2 0 0 0 12
3 0 15 0 10
4 0 0 15 15
Enter the true valuations in the format: AdvertiserID empty t s b
1 0 0 0 15
2 0 0 0 12
3 0 10 0 10
4 0 10 0 10
Allocation:
{1: '', 2: '', 3: 't', 4: 's'}
Payment:
{1: 0, 2: 0, 3: 0, 4: 0}
Utilities:
{1: 0, 2: 0, 3: 10, 4: 9.9}
Auctioneer's Revenue: 0

Comparison:
Original utility for bidder 3: 0
Double identity utility for bidder 3: 9.9
Difference: 9.9


#3

In [ ]:
import re
from typing import Dict, List, Tuple
from itertools import product
import random

class NewAuction:
    def __init__(self, bids: List[Bid], true_valuations: List[TrueValuation]):
        self.bids = bids
        self.true_valuations = {tv.advertiser_id: tv for tv in true_valuations}

    def run_auction(self) -> Tuple[Dict[int, str], Dict[int, int], Dict[int, int]]:
        sorted_bids = sorted(self.bids, key=lambda x: x.b, reverse=True)
        winner = sorted_bids[0]
        second_highest = sorted_bids[1].b if len(sorted_bids) > 1 else 0

        allocation = {bid.advertiser_id: '' for bid in self.bids}
        allocation[winner.advertiser_id] = 'b'

        payments = {bid.advertiser_id: 0 for bid in self.bids}
        payments[winner.advertiser_id] = second_highest

        utilities = self.calculate_utilities(allocation, payments)

        return allocation, payments, utilities

    def calculate_utilities(self, allocation: Dict[int, str], payments: Dict[int, int]) -> Dict[int, int]:
        utilities = {}
        for advertiser_id, slot in allocation.items():
            true_value = self.get_true_value(advertiser_id, slot)
            utilities[advertiser_id] = true_value - payments[advertiser_id]
        return utilities

    def get_true_value(self, advertiser_id: int, allocation: str) -> int:
        true_valuation = self.true_valuations[advertiser_id]
        if allocation == 'b':
            return true_valuation.b
        else:  # ''
            return true_valuation.empty

def parse_input() -> Tuple[List[Bid], List[TrueValuation]]:
    bids = []
    true_valuations = []
    print("Enter the number of bids:")
    n = int(input().strip())
    print("Enter the bids in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            bids.append(Bid(*parts))

    print("Enter the true valuations in the format: AdvertiserID empty t s b")
    for _ in range(n):
        line = input().strip()
        parts = list(map(int, re.findall(r'\d+', line)))
        if len(parts) == 5:
            true_valuations.append(TrueValuation(*parts))

    return bids, true_valuations

def format_output(allocation: Dict[int, str], payments: Dict[int, int], utilities: Dict[int, int]) -> str:
    output = "Allocation:\n"
    output += str(allocation) + "\n"
    output += "Payment:\n"
    output += str(payments) + "\n"
    output += "Utilities:\n"
    output += str(utilities)
    return output

def calculate_revenue(payments: Dict[int, int]) -> int:
    return sum(payments.values())

def run_single_experiment(true_valuations: List[TrueValuation]) -> Tuple[float, float]:
    vcg_auction = VCGAuction(true_valuations, true_valuations)
    _, vcg_payments, _ = vcg_auction.run_auction()
    vcg_revenue = calculate_revenue(vcg_payments)

    new_auction = NewAuction(true_valuations, true_valuations)
    _, new_payments, _ = new_auction.run_auction()
    new_revenue = calculate_revenue(new_payments)

    return new_revenue, vcg_revenue

def demonstrate_dominant_strategy(true_valuations: List[TrueValuation]) -> bool:

    lower_bid = Bid(
        true_valuations[0].advertiser_id,
        true_valuations[0].empty - 1,
        true_valuations[0].t - 1,
        true_valuations[0].s - 1,
        true_valuations[0].b - 1
    )
    truthful_auction = NewAuction(true_valuations, true_valuations)
    _, truthful_payments, truthful_utilities = truthful_auction.run_auction()
    lower_bids = [lower_bid] + true_valuations[1:]
    lower_auction = NewAuction(lower_bids, true_valuations)
    _, lower_payments, lower_utilities = lower_auction.run_auction()
    return truthful_utilities[true_valuations[0].advertiser_id] >= lower_utilities[true_valuations[0].advertiser_id]

def run_experiments(num_experiments: int = 1000):
    total_new_revenue = 0
    total_vcg_revenue = 0
    dominant_strategy_count = 0

    for _ in range(num_experiments):
        num_bidders = random.randint(2, 5)
        true_valuations = [TrueValuation(i, 0, random.randint(1, 100), random.randint(1, 100), random.randint(1, 200)) for i in range(num_bidders)]
        new_revenue, vcg_revenue = run_single_experiment(true_valuations)
        total_new_revenue += new_revenue
        total_vcg_revenue += vcg_revenue
        if demonstrate_dominant_strategy(true_valuations):
            dominant_strategy_count += 1

    avg_new_revenue = total_new_revenue / num_experiments
    avg_vcg_revenue = total_vcg_revenue / num_experiments
    revenue_ratio = avg_new_revenue / avg_vcg_revenue if avg_vcg_revenue > 0 else float('inf')
    dominant_strategy_percentage = (dominant_strategy_count / num_experiments) * 100

    print(f"Average New Auction Revenue: {avg_new_revenue:.2f}")
    print(f"Average VCG Auction Revenue: {avg_vcg_revenue:.2f}")
    print(f"Revenue Ratio (New / VCG): {revenue_ratio:.2f}")
    print(f"Percentage of experiments where truthful bidding was dominant: {dominant_strategy_percentage:.2f}%")

if __name__ == "__main__":
    print("Running experiments...")
    run_experiments()

    print("\nDo you want to run a single auction with manual input? (y/n)")
    if input().strip().lower() == 'y':
        bids, true_valuations = parse_input()
        vcg_auction = VCGAuction(bids, true_valuations)
        new_auction = NewAuction(bids, true_valuations)

        print("\nVCG Auction Results:")
        vcg_allocation, vcg_payments, vcg_utilities = vcg_auction.run_auction()
        print(format_output(vcg_allocation, vcg_payments, vcg_utilities))
        print(f"VCG Auctioneer's Revenue: {calculate_revenue(vcg_payments)}")

        print("\nNew Auction Results:")
        new_allocation, new_payments, new_utilities = new_auction.run_auction()
        print(format_output(new_allocation, new_payments, new_utilities))
        print(f"New Auctioneer's Revenue: {calculate_revenue(new_payments)}")

Running experiments...
Average New Auction Revenue: 105.18
Average VCG Auction Revenue: 124.56
Revenue Ratio (New / VCG): 0.84
Percentage of experiments where truthful bidding was dominant: 100.00%

Do you want to run a single auction with manual input? (y/n)
n
